<a href="https://colab.research.google.com/github/kenanmorani/Audio_Signal_Processing_For_Music_Applications_idu/blob/main/Pitch%20Exctraction/HW3_audio_signal_processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# HW3 Audio Signal Processing for Music Applications course
Izmir Democracy University, Electrical and Electronics Program    
Submitted to Professor Barsi Bozkurt      
Submitted by Kenan MORANI     

# Installing and importing the required libraries

In [1]:
!pip install essentia mir_eval pysoundfile

     |████████████████████████████████| 12.0MB 248kB/s 
     |████████████████████████████████| 92kB 9.3MB/s 
  Created wheel for mir-eval: filename=mir_eval-0.6-cp36-none-any.whl size=96515 sha256=d653e64d72d0d70e24e6cc8b301e2c8f2f789ece615f4a1da277a586c9ad0c50
  Stored in directory: /root/.cache/pip/wheels/49/ce/30/730fa72addf275e49d90683b01b3613048b4be3bf7ff8eb6ec
Successfully built mir-eval


In [2]:
import os
from essentia import *
from essentia.standard import *
import matplotlib.pyplot as plt
import numpy as np
from scipy.signal import get_window
import soundfile as sf
import urllib.request
from IPython.display import Audio
import mir_eval

In [3]:
#Downloading a standard mir dataset
!pip install mirdata
import mirdata
orchset = mirdata.initialize('orchset')
orchset.download()

     |████████████████████████████████| 6.1MB 3.6MB/s 
     |████████████████████████████████| 184kB 41.6MB/s 
     |████████████████████████████████| 61kB 6.1MB/s 
     |████████████████████████████████| 5.6MB 29.8MB/s 
     |████████████████████████████████| 61kB 8.0MB/s 
     |████████████████████████████████| 61kB 7.6MB/s 
     |████████████████████████████████| 61kB 7.7MB/s 
  Created wheel for librosa: filename=librosa-0.8.0-cp36-none-any.whl size=201375 sha256=8fa68508fa55260f27ef74927e13707bac078eaaffc75934aa17fcad10b823f2
  Stored in directory: /root/.cache/pip/wheels/ee/10/1e/382bb4369e189938d5c02e06d10c651817da8d485bfd1647c9
  Created wheel for jams: filename=jams-0.3.4-cp36-none-any.whl size=64925 sha256=fceee20ed44c3170bd3fa64c428a62d509dfd6b20ee491af2dd79fb8f85d1d87
  Stored in directory: /root/.cache/pip/wheels/2b/b8/c7/092096a3af0eb864cca4e79a44bc883dee5b9767da2c6f8ab3
  Created wheel for pretty-midi: filename=pretty_midi-0.2.9-cp36-none-any.whl size=5591954 sha256=0880

INFO: Downloading ['all'] to /root/mir_datasets/orchset
INFO: [all] downloading Orchset_dataset_0.zip
311MB [00:45, 7.15MB/s]                           


# Exctracting f0s

In [4]:
#Collecting audio file names 
all_files = []
for root, dirs, files in os.walk('/root/mir_datasets/orchset/audio/mono/'):
    for file in files:
        if file.endswith('.wav'):
            file_name = os.path.join(root,file)
            all_files.append(file_name)
# Exctracting f0s            
for file in all_files:
 x, fs = sf.read(file)
 x = x / np.max(np.abs(x))#amplitude normalisation
 minF0 = 50 #in Hz
 maxF0 = 2000 #in Hz
 windowSize = 4096
 hopSize = 1024
 w = get_window('blackman', windowSize)
 startIndexes = np.arange(0, x.size - windowSize, hopSize, dtype = int)
 numWindows = startIndexes.size
 Audio(x, rate=fs)
 run_predominant_melody = PitchMelodia(guessUnvoiced = True,
                                      frameSize = windowSize,
                                      hopSize = hopSize,
                                      maxFrequency = maxF0,
                                      minFrequency = minF0,
                                      sampleRate = fs);
 f0, confidence = run_predominant_melody(x)

# Using essentia-melodia algorithm

In [5]:
# After deleting Readme.txt in orchest folder
os.remove('/root/mir_datasets/orchset/README.txt')

In [8]:
scoress = [] 
for root, dirs, files in os.walk('/root/mir_datasets/orchset/'):
    for file in files:
        #Collecting Annotations
        if file.endswith('.mel'):
            file_name = os.path.join(root,file)
            all_midi.append(file_name)        
            ref_time, ref_freq = mir_eval.io.load_time_series(file)
        #Working on estimations
        elif file.endswith('.wav'):
            est_time, est_freq = mir_eval.io.load_time_series(file)
             # Evaluating the results
            for score in scores:
              scoress = mir_eval.melody.evaluate(ref_time, ref_freq,est_time, est_freq)
              print(score,'\t',scoress[score])


FileNotFoundError: ignored

# Using crepe algorithm

In [9]:
!pip install crepe
import crepe

     |████████████████████████████████| 368kB 6.2MB/s 
  Created wheel for crepe: filename=crepe-0.0.11-cp36-none-any.whl size=134848478 sha256=1112a570a4265d8547798e0aeb9f9a3338110aec6895298db854823b97f71320
  Stored in directory: /root/.cache/pip/wheels/7f/68/d0/502d27bfe590bfa51c6fe95cf9e4482fed18c22c480a8d7c77
Successfully built crepe


In [11]:
scores =[]
for root, dirs, files in os.walk('/root/mir_datasets/orchset/'):
        if file.endswith('.mel'):
            file_name = os.path.join(root,file)
            all_midi.append(file_name)        
            #Reading ground-truth
            ref_time, ref_freq = mir_eval.io.load_time_series(file)
        elif file.endswith('.wav'):
           hopSize = 441#10 miliseconds
           windowSize = 4096
           fs = 44100
           x, fs = sf.read(file)
           x = x / np.max(np.abs(x))#amplitude normalisation
           #x = EqualLoudness()(x)
           est_time, est_freq, confidence, activation = crepe.predict(x, fs,viterbi=True)
           #Running evaluation and ptinting
           for score in scores:
             scores[score] = mir_eval.melody.evaluate(ref_time, ref_freq,est_time, est_freq)
             print(score,'\t',scores[file])

RuntimeError: ignored